In [ ]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from os import path
import librosa
import librosa.display
from sklearn.model_selection import train_test_split

import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing

import cv2

In [ ]:
def resize_all(images_2d, wanted_shape):
    res_images = []
    for image in images_2d:
        resized = cv2.resize(image[0], dsize=wanted_shape)
        #print(resized.shape)
        res_images.append([resized])
    return res_images

def normalize_and_pca(images_2d, variance, normalize_only=False, pca_only=False):
    orig_x = images_2d[0][0].shape[0]
    orig_y = images_2d[0][0].shape[1]
    image_length = orig_x * orig_y
    images_1d = []
    
    for image in images_2d:
        images_1d.append(image[0].reshape((image_length)))
    
    data_encoded = images_1d
    
    # normalize
    if not pca_only:
        data_encoded = preprocessing.normalize(data_encoded)

        if normalize_only:
            images_2d = []
            for i in range(len(data_encoded)):
                images_2d.append([data_encoded[i].reshape(orig_x, orig_y)])
            return images_2d
    
    # pca
    pca = PCA(n_components=variance) # Used instead of k to compute minimum number of dimensions to preserve % of variance
    X_reduced = pca.fit(data_encoded).transform(data_encoded)
    X_recovered = pca.inverse_transform(X_reduced)
    
    images_2d = []
    for i in range(len(X_recovered)):
        images_2d.append([X_recovered[i].reshape(orig_x, orig_y)])
    return images_2d
    
def mel_spect(filepath):
    y, sr = librosa.load(filepath)
    mel_spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
    mel_spect = librosa.power_to_db(mel_spect, ref=np.max)
    #librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
    return mel_spect

## GTZAN

In [ ]:
X = []
labels = []
dir_name = 'genres/'
for path in Path(dir_name).rglob('*.wav'):
    x = mel_spect(path)

    y_padding = 660 - x.shape[1]
    d = np.zeros((128, y_padding))
    x = np.hstack((x, d))

    print(x.shape)
    assert(x.shape == (128, 660))

    X.append(np.array([x]))
    labels.append(path.name.split('.')[0])

# Y to int label
Y = []
unique_labels = list(np.unique(labels))
for i, y in enumerate(labels):
    Y.append(unique_labels.index(y))

## FMA

In [ ]:
X = []
labels = []
dir_name = 'raw_data/fma/fma_medium'

try:
    for path in Path(dir_name).rglob('*.wav'):
        print(path)
        try:
            x = mel_spect(path)

            y_padding = 660 - x.shape[1]
            d = np.zeros((128, y_padding))
            x = np.hstack((x, d))

            #print(x.shape)
            assert(x.shape == (128, 660))

            X.append(np.array([x]))
            labels.append(path.name.split('.')[0])
        except Exception as e:
            print('failed')
            print(e)
            print(path)

except KeyboardInterrupt:
    # Y to int label
    Y = []
    unique_labels = list(np.unique(labels))
    for i, y in enumerate(labels):
        Y.append(unique_labels.index(y))

## RAVDESS

In [ ]:
data = None
with open('raw_data/design_matrix.tab') as players_data:
    data = players_data.read()
    
data = data.split('\t')

label_nums = []

for i, d in enumerate(data):
    if i > 7 and i % 7 == 0:
        label_nums.append(int(d.split('\n')[0]))
        
X = []
labels = []
dir_name = 'raw_data/ravdess/'
for i, path in enumerate(Path(dir_name).rglob('*.wav')):
    x = mel_spect(path)

    if x.shape[1] > 660:
        print(x.shape)
        continue
    
    y_padding = 660 - x.shape[1]
    d = np.zeros((128, y_padding))
    x = np.hstack((x, d))

    assert(x.shape == (128, 660))

    X.append(np.array([x]))
    labels.append(label_nums[i])
    
Y = labels

## PMC

In [ ]:
X = []
labels = []
dir_name = 'raw_data/stimuli/test-stimuli-200-2009-05-29/'
for i, path in enumerate(Path(dir_name).rglob('*.wav')):
    x = mel_spect(path)

    if x.shape[1] > 660:
        print(x.shape)
        continue
    
    y_padding = 660 - x.shape[1]
    d = np.zeros((128, y_padding))
    x = np.hstack((x, d))

    assert(x.shape == (128, 660))

    X.append(np.array([x]))
    labels.append(label_nums[i])
    
Y = [l - 1 for l in labels]

## Preprocess and Save

In [ ]:
X_resized = resize_all(X, (165, 32))
X_pca = normalize_and_pca(np.array(X_resized), variance=0.3, normalize_only=True)

X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=42)

print(len(X_train))
print(len(X_test))

train = []
for i in range(len(X_train)):
    sample = (X_train[i], y_train[i])
    train.append(sample)
    
test = []
for i in range(len(X_test)):
    sample = (X_test[i], y_test[i])
    test.append(sample)
    
pickle.dump(train, open('data/GTZAN_resized.train', 'wb'))
pickle.dump(test, open('data/GTZAN_resized.test', 'wb'))

In [ ]:
plt.imshow(train[50][0][0], interpolation='nearest')
plt.show()